In [ ]:
!pip install azure-cosmos
!pip install fastapi
!pip install colabcode

In [ ]:
from os import getenv
import json
import pandas as pd
import uuid

from azure.cosmos import CosmosClient, PartitionKey
import azure.cosmos.errors as errors
import azure.cosmos.documents as documents
import azure.cosmos.http_constants as http_constants

from fastapi import FastAPI

In [ ]:
CONFIG = {
    "endpoint": "https://db-nosql.documents.azure.com:443/",
    "database" : "meubndes",
    "container" : "operacoes",
    "readkey" : "Q9ikAx2k6puQIrVHHN1xosrfAF0amUqTmBC724wJdNxTP3JGNKnQMpixOOkHhgWiVyCMi4xZup7rMeetig1eWg=="
}

CONTAINER_LINK = f"dbs/{CONFIG['database']}/colls/{CONFIG['container']}"
FEEDOPTIONS = {}
FEEDOPTIONS["enableCrossPartitionQuery"] = True
# There is also a partitionKey Feed Option, but I was unable to figure out how to us it.

In [ ]:
QUERY = {
    "query": f"SELECT * from c"
}

In [ ]:
# Initialize the Cosmos client
client = CosmosClient(
    CONFIG["endpoint"], {"masterKey": CONFIG["readkey"]}
)

# Create a database if not exists
DATABASE = client.create_database_if_not_exists(id=CONFIG['database'])

# Create a collection of items in a Container if not exists
CONTAINER = DATABASE.create_container_if_not_exists(
    id=CONFIG['container'],
    partition_key=PartitionKey(path='/id'),
    offer_throughput=400
)

In [ ]:
def get_query_items(query=None):
    params = {
      'query': query,
      'enable_cross_partition_query': True
    }
    res = CONTAINER.query_items(**params)
    return list(res)

def get_items(item_id=None):
    query = 'SELECT * FROM c'
    if item_id:
        query += ' WHERE c.id = @item_id'

    params = {
        'query': query,
        'parameters': [{"name": "@item_id", "value": str(item_id)}],
        'enable_cross_partition_query': True
    }
    res = CONTAINER.query_items(**params)
    return list(res)


def create_item(item):
    item['id'] = str(uuid.uuid4())
    CONTAINER.create_item(body=item)
    return item


def delete_item(item_id):
    res = get_items(item_id)
    if not res:
        return False
    item = res[0]
    CONTAINER.delete_item(item, partition_key=item['category'])
    return True


def complete_item(item_id, is_complete=True):
    res = get_items(item_id)
    if not res:
        return False
    item = res[0]
    new_item = item
    new_item['isComplete'] = is_complete
    CONTAINER.replace_item(item, new_item)
    return True

In [ ]:
print(get_query_items('select top 1 * from c where c.municipio_codigo = 3106200'))

[{'id': 'a70f9500-70f9-4b10-9bf4-16557c068e6e', 'cliente': 'FLEXLINE INDUSTRIA E COMERCIO LTDA', 'cpf_cnpj': '**.*29.143/0001-**', 'uf': ' MG', 'municipio': 'BELO HORIZONTE', 'municipio_codigo': 3106200, 'data_da_contratacao': '2002-01-10', 'valor_da_operacao_em_reais': 28776, 'valor_desembolsado_reais': 28776.0, 'fonte_de_recurso_desembolsos': 'RECURSOS LIVRES - PRÓPRIOS', 'custo_financeiro': 'TJLP', 'juros': 5.0, 'prazo_carencia_meses': 6, 'prazo_amortizacao_meses': 54, 'modalidade_de_apoio': 'REEMBOLSÁVEL', 'forma_de_apoio': 'INDIRETA', 'produto': 'BNDES FINAME', 'instrumento_financeiro': 'OUTROS', 'inovacao': 'NÃO', 'area_operacional': 'AREA DE OPERACOES E CANAIS DIGITAIS', 'setor_cnae': 'COMERCIO E SERVICOS', 'subsetor_cnae_agrupado': 'Comércio', 'subsetor_cnae_codigo': 'G4754700', 'subsetor_cnae_nome': 'COM VAREJISTA ESPECI MOVEIS, COLCHOARIA E ARTIGOS DE ILUMINACAO', 'setor_bndes': 'COMERCIO/SERVICOS', 'subsetor_bndes': 'COMÉRCIO E SERVIÇOS', 'porte_do_cliente': 'PEQUENA', 'natu

In [ ]:
app = FastAPI()

@app.on_event("startup")
def load_model():
    global query, model, transformation
    #fpickle = pickle.load(open("model_tree.pkl", "rb"))
    #model = fpickle['model']
    #transformation = fpickle['transformation']

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}

@app.get('/zip/{item_code}')
def get_data_by_zip(item_code: int):
    query = "select top 1 * from c where c.municipio_codigo = {}".format(str(item_code))
    print(query)
    result = get_query_items(query)
    return result

In [ ]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [ ]:
server.run_app(app=app)

INFO:     Started server process [342]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://db84-35-229-160-204.ngrok.io" -> "http://localhost:10000"
INFO:     201.80.0.251:0 - "GET / HTTP/1.1" 200 OK
INFO:     201.80.0.251:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     201.80.0.251:0 - "GET /zip/ HTTP/1.1" 404 Not Found
select top 1 * from c where c.municipio_codigo = 5200050
INFO:     201.80.0.251:0 - "GET /zip/5200050 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [342]
